<a href="https://colab.research.google.com/github/InjunSeo/ai/blob/master/AI_13_%EC%84%9C%EC%9D%B8%EC%A4%80_Session1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Requirements
* **요구1** 지역에 따라서 선호하는 게임 장르가 다른지.
* **요구2** 연도별 게임의 트렌드가 있는지 여부 파악. 
* **요구3** 출고량이 높은 게임에 대한 분석 및 시각화 프로세스 포함.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import io
import pandas as pd

url = '/content/drive/MyDrive/Data/vgames2.csv'

In [120]:
data = pd.read_csv(url)
data.head()

,Unnamed: 0,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


# 데이터에 대한 전반적 정보 확인

## 제공된 feature description
1. Name : 게임 이름
2. Platform : 게임이 지원되는 플랫폼의 이름
  * '2600'의 경우, '아타리 2600'인 것으로 추측된다. https://ko.wikipedia.org/wiki/%EC%95%84%ED%83%80%EB%A6%AC_2600
3. Year : 게임이 출시된 연도.
4. Genre : 게임의 장르.
5. Publisher : 게임 제작한 회사.
6. NA_Sales : 북미지역에서의 출고량
7. EU_Sales : 유럽지역에서의 출고량.
8. JP_Sales : 일본지역에서의 출고량.
9. Other_Sales : 기타지역에서의 출고량

In [123]:
print(data.shape)

data.info()
 # => type 변환 필요: (1) Year: float -> int (2) 출고량 -> float 또는 int로
 # 결측치 : year, Genre, Publisher 

data['Genre'].unique()
data['Platform'].unique()

(16598, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   16598 non-null  int64  
 1   Name         16598 non-null  object 
 2   Platform     16598 non-null  object 
 3   Year         16327 non-null  float64
 4   Genre        16548 non-null  object 
 5   Publisher    16540 non-null  object 
 6   NA_Sales     16598 non-null  object 
 7   EU_Sales     16598 non-null  object 
 8   JP_Sales     16598 non-null  object 
 9   Other_Sales  16598 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 1.3+ MB


array(['DS', 'Wii', 'PSP', 'PS3', 'PC', 'PS', 'GBA', 'PS4', 'PS2', 'XB',
       'X360', 'GC', '3DS', '2600', 'SAT', 'GB', 'NES', 'DC', 'N64',
       'XOne', 'SNES', 'WiiU', 'PSV', 'GEN', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

In [124]:
del data['Unnamed: 0']
data

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04
...,...,...,...,...,...,...,...,...,...
16593,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0,0.01
16594,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0,0,0
16595,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16596,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


# 요구1: 지역별 선호하는 게임 장르가 있는가?

## 필요 데이터: 지역별 출고량(sales)

In [125]:
df = data.copy()
df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [126]:
df.loc[130:160]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
130,Hot Wheels: Battle Force 5,DS,2009.0,Racing,Activision,0.13,0.02,0,0.01
131,Call of Duty 4: Modern Warfare,PC,2007.0,Shooter,Activision,0,1.12,0,0.03
132,Rabbids Invasion: The Interactive TV Show,PS4,2014.0,Misc,Ubisoft,0,0.01,0,0
133,Cabela's Big Game Hunter: Pro Hunts,WiiU,2014.0,Shooter,Activision,0.02,0,0,0
134,Forza Horizon 2,X360,2014.0,Racing,Microsoft Game Studios,0.35,0.43,0,0.07
135,Hitman: HD Trilogy,PS3,2013.0,Action,Square Enix,0.07,0.1,0,0.04
136,Super Robot Taisen XO,X360,2006.0,Strategy,Banpresto,0,0,0.01,0
137,Call of Juarez: The Cartel,PS3,2011.0,Shooter,Ubisoft,0.12,0.13,0,0.05
138,Warriors Orochi 3,PS4,2014.0,Action,Tecmo Koei,0.04,50K,0.04,0.02
139,The Incredibles: Rise of the Underminer,PS2,2005.0,Action,THQ,0.18,0.14,0,0.05


## Sales 데이터 수정
* 비표준적 데이터 수정 필요
* 가정: 출고량 데이터들의 단위는 백만(millions)인 것 같다.   
  * 왜냐하면, 일부 데이터들에 'M', 'K'들이 붙어있는데 'M'과 'K'는 각각 'millions'와 'thousand'를 의미한다. 그런데 출고량 컬럼들은 '북미', '유럽' 등의 출고량을 의미한다. 따라서 단위가 '천'이 아니라 '백만'일 것이다.   
1. 'M' 붙은 데이터에서 'M' 삭제  
    e.g., 480K -> 480000   
2. 'K' 붙은 데이터에서 'K' 삭제  
    e.g., 0.1M -> 0.1 * 100 0000   
3. object => int로 변경

In [127]:
def convertQuantity(sales):
  if sales[-1] == 'K':      # '480K'
    result = sales.replace('K', '000')  # '480000'
  elif sales[-1] == 'M':     # '0.1M'
    result = sales.replace('M', '')   # '0.1'
    result = float(result)             # 0.1
    result *= (10**6)         #  
  else:                     # '0.17'
    result = float(sales)            # 0.17
    result *= (10**6)  
    
  return int(result)

In [128]:
df['NA_Sales'] = df['NA_Sales'].apply(convertQuantity)
df['EU_Sales'] = df['EU_Sales'].apply(convertQuantity)
df['JP_Sales'] = df['JP_Sales'].apply(convertQuantity)
df['Other_Sales'] = df['Other_Sales'].apply(convertQuantity)

In [129]:
df.loc[130:160]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
130,Hot Wheels: Battle Force 5,DS,2009.0,Racing,Activision,130000,20000,0,10000
131,Call of Duty 4: Modern Warfare,PC,2007.0,Shooter,Activision,0,1120000,0,30000
132,Rabbids Invasion: The Interactive TV Show,PS4,2014.0,Misc,Ubisoft,0,10000,0,0
133,Cabela's Big Game Hunter: Pro Hunts,WiiU,2014.0,Shooter,Activision,20000,0,0,0
134,Forza Horizon 2,X360,2014.0,Racing,Microsoft Game Studios,350000,430000,0,70000
135,Hitman: HD Trilogy,PS3,2013.0,Action,Square Enix,70000,100000,0,40000
136,Super Robot Taisen XO,X360,2006.0,Strategy,Banpresto,0,0,10000,0
137,Call of Juarez: The Cartel,PS3,2011.0,Shooter,Ubisoft,120000,130000,0,50000
138,Warriors Orochi 3,PS4,2014.0,Action,Tecmo Koei,40000,50000,40000,20000
139,The Incredibles: Rise of the Underminer,PS2,2005.0,Action,THQ,180000,140000,0,50000


# 요구2: 연도별 게임 트랜드 파악
## 'Year' 컬럼의 값들이 필요  
  1. Year 컬럼의 결측치 또는 '0.0'을 제거
  2. Year 컬럼의 비표준적 데이터들 수정

## Year 컬럼의 결측치 제거

In [130]:
import numpy as np
df1 = data.copy()

In [131]:
df1['Year'].fillna(np.nan, inplace=True)
df1.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [132]:
df1.dropna(inplace=True)

In [133]:
df1.info()
df1.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16241 entries, 0 to 16597
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16241 non-null  object 
 1   Platform     16241 non-null  object 
 2   Year         16241 non-null  float64
 3   Genre        16241 non-null  object 
 4   Publisher    16241 non-null  object 
 5   NA_Sales     16241 non-null  object 
 6   EU_Sales     16241 non-null  object 
 7   JP_Sales     16241 non-null  object 
 8   Other_Sales  16241 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.2+ MB


(16241, 9)

## Year 값이 0.0인 데이터 제거

In [134]:
df1[df1['Year'] == 0.0]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
9276,Trade & Battle: Card Hero,GB,0.0,Strategy,Nintendo,0,0,0.2,0
14666,Strider 2,PS,0.0,Platform,Virgin Interactive,0.05,0.03,0,0.01
16394,Point Blank 3,PS,0.0,Shooter,Namco Bandai Games,0.01,0.01,0,0


In [135]:
filt = df1['Year'] == 0.0
df1[filt].index

Int64Index([9276, 14666, 16394], dtype='int64')

In [136]:
df1.drop(index = df1[filt].index, inplace=True)

In [137]:
df1.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


## Year: int로 

In [138]:
df1['Year'] = df1['Year'].astype(int)
df1.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010,Platform,D3Publisher,0.12,0.09,0,0.04


## 연도 표기가 비표준적인 데이터 수정   
e.x. 9, 98, 11 등      
* Case1:  '19'뺀 애들: 94, 98, 99
* Case2: '20' 뺀 애들: 11   
* str로 변환해주기   
  * Case1) '19' 붙여주기   
  * Case2) '20' 또는 '200' 붙여주기

In [100]:
df1.loc[74]

Unnamed: 0                                          75
Name           Age of Empires III: Complete Collection
Platform                                            PC
Year                                                 9
Genre                                         Strategy
Publisher                       Microsoft Game Studios
NA_Sales                                          0.01
EU_Sales                                          0.12
JP_Sales                                             0
Other_Sales                                       0.03
Name: 74, dtype: object

In [ ]:
filt3 = df1['Year'] > 0
filt4 = df1['Year'] < 10

df1[filt3 & filt4].index

Int64Index([   74,   768,  1015,  1220,  1364,  1409,  1684,  2249,  2454,
             2467,  2548,  2843,  3108,  3746,  3879,  4076,  4746,  4800,
             5314,  6130,  7055,  7178,  7362,  7600,  7845,  8144,  8509,
             8583,  8681,  8765,  8912,  8928,  9080,  9326,  9890, 10357,
            10406, 11163, 11236, 12495, 12636, 13739, 14161, 15260, 15263,
            15442, 15511, 15612, 16103, 16149],
           dtype='int64')

In [ ]:
filt5 = df1['Year'] >= 10
filt6 = df1['Year'] < 23
df1[filt5 & filt6].index

Int64Index([  213,   304,  3020,  3044,  4010,  4319,  4727,  5193,  5946,
             6113,  6159,  6192,  6899,  7184,  7555,  7627,  8191,  8550,
             8839,  9291, 10405, 10621, 10811, 10922, 11263, 11519, 11960,
            12382, 12869, 15245, 15447, 15488, 15836],
           dtype='int64')

In [ ]:
# Year == '98' 등인 row의 index
df1[(df1['Year'] > 60 ) & (df1['Year'] <= 99)].index

Int64Index([   81,   906,  2429,  3492,  4365,  4807,  4839, 10315, 10705,
            11334, 14572, 14785, 15985, 16482],
           dtype='int64')

In [139]:
# str로 바꿔준 후, 앞에 '19' 또는 '20' 붙이기
def fixYear(year):
  if year >0 and year < 10:
    return '200' + str(year)
  elif year >= 10 and year < 23:
    return '20' + str(year)
  elif year > 60 and year <= 99:
    return '19' + str(year)  
  return year

df1['Year'] = df1['Year'].apply(fixYear)

In [140]:
df1['Year'] = df1['Year'].astype(int)

# df1.loc[[74,  768,  1015,  1220,  1364,  1409,  1684,  2249,  2454,
#              2467,  2548,  2843,  3108,  3746,  3879,  4076,  4746,  4800,
#              5314,  6130,  7055,  7178,  7362,  7600,  7845,  8144,  8509,
#              8583,  8681,  8765,  8912,  8928,  9080,  9326,  9890, 10357,
#             10406, 11163, 11236, 12495, 12636, 13739, 14161, 15260, 15263,
#             15442, 15511, 15612, 16103, 16149]]
# df1.loc[[213,   304,  3020,  3044,  4010,  4319,  4727,  5193,  5946,
#              6113,  6159,  6192,  6899,  7184,  7555,  7627,  8191,  8550,
#              8839,  9291, 10405, 10621, 10811, 10922, 11263, 11519, 11960,
#             12382, 12869, 15245, 15447, 15488, 15836]]
# df1.loc[[81,   906,  2429,  3492,  4365,  4807,  4839, 10315, 10705,
#             11334, 14572, 14785, 15985, 16482]]

df1.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010,Platform,D3Publisher,0.12,0.09,0,0.04
